# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Grabbing the document 
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Norman Wells,65.2820,-126.8329,-25.60,75,1,2.30,CA,1613279896
1,Touros,-5.1989,-35.4608,77.00,88,40,3.44,BR,1613279896
2,Qaanaaq,77.4840,-69.3632,9.48,78,0,4.90,GL,1613279251
3,Punta Arenas,-53.1500,-70.9167,42.80,81,75,14.97,CL,1613279755
4,Ushuaia,-54.8000,-68.3000,41.00,81,40,11.50,AR,1613279896
...,...,...,...,...,...,...,...,...,...
544,Ayagoz,47.9714,80.4392,20.30,90,0,4.03,KZ,1613280830
545,Matagami,49.7502,-77.6328,-5.22,92,100,4.23,CA,1613280830
546,Braintree,51.8782,0.5529,32.00,71,100,5.01,GB,1613280830
547,Yeppoon,-23.1333,150.7333,93.20,33,90,8.05,AU,1613280830


In [ ]:
# Making the df smaller for testing 
#weather_data = weather_data.head(50)
#weather_data.dtypes

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]]

# Convert Humidity to float and store
humidity_rate = weather_data["Humidity"]

# Store city names
location_names = weather_data["City"]

In [10]:
# Create a humidity heatmap layer, https://jupyter-gmaps.readthedocs.io/en/latest/api.html
fig = gmaps.figure(center=(51.4779, 0.0015), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Searching for ideal weather conditions:  
# A max temp: lower than 80 degrees but higher than 70. 
ideal_weather_data = weather_data.loc[(weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)]

# Wind speed less than 10 mph. 
ideal_weather_data = ideal_weather_data.loc[ideal_weather_data["Wind Speed"] <= 10]

# Light cloudiness. 
ideal_weather_data = ideal_weather_data.loc[ideal_weather_data["Cloudiness"] <= 20]

ideal_weather_data.shape
# No null values pulled in this df 

(30, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Saving df and adding column 
hotel_df = ideal_weather_data[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = "" 

# Making it small for testing
# hotel_df = hotel_df.head(10)
hotel_df

<ipython-input-13-22ec98c0114f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
14,Kahului,US,20.8947,-156.4700,
25,Kapaa,US,22.0752,-159.3190,
52,New Norfolk,AU,-42.7826,147.0587,
55,Benguela,AO,-12.5763,13.4055,
75,Bubaque,GW,11.2833,-15.8333,
94,Harnai,PK,30.1005,67.9374,
105,Raglan,NZ,-37.8000,174.8833,
132,Yaan,NG,7.3833,8.5667,
173,Batemans Bay,AU,-35.7167,150.1833,
203,Monrovia,LR,6.3005,-10.7969,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "radius": 5000, 
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat/lng from df
    lat_lng = f"{row['Lat']},{row['Lng']}"

    # add keyword to params dict
    params["location"] = lat_lng

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {lat_lng}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {lat_lng} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 14: 20.8947,-156.47.
Closest hotel to 20.8947,-156.47 is Maui Seaside Hotel.
------------
Retrieving Results for Index 25: 22.0752,-159.319.


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel to 22.0752,-159.319 is Sheraton Kauai Resort at Coconut Beach.
------------
Retrieving Results for Index 52: -42.7826,147.0587.
Closest hotel to -42.7826,147.0587 is The Shingles Riverside Cottages.
------------
Retrieving Results for Index 55: -12.5763,13.4055.
Closest hotel to -12.5763,13.4055 is Hotel Praia Morena.
------------
Retrieving Results for Index 75: 11.2833,-15.8333.
Closest hotel to 11.2833,-15.8333 is Casa Dora, Bubaque.
------------
Retrieving Results for Index 94: 30.1005,67.9374.
Closest hotel to 30.1005,67.9374 is Shah g hotel.
------------
Retrieving Results for Index 105: -37.8,174.8833.
Closest hotel to -37.8,174.8833 is Raglan Sunset Motel.
------------
Retrieving Results for Index 132: 7.3833,8.5667.
Missing field/result... skipping.
------------
Retrieving Results for Index 173: -35.7167,150.1833.
Closest hotel to -35.7167,150.1833 is Batemans Bay Marina Resort.
------------
Retrieving Results for Index 203: 6.3005,-10.7969.
Closest hotel to 6.30

In [15]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel
25,Kapaa,US,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
52,New Norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
55,Benguela,AO,-12.5763,13.4055,Hotel Praia Morena
75,Bubaque,GW,11.2833,-15.8333,"Casa Dora, Bubaque"
94,Harnai,PK,30.1005,67.9374,Shah g hotel
105,Raglan,NZ,-37.8000,174.8833,Raglan Sunset Motel
132,Yaan,NG,7.3833,8.5667,
173,Batemans Bay,AU,-35.7167,150.1833,Batemans Bay Marina Resort
203,Monrovia,LR,6.3005,-10.7969,Sunset Inn


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))